# Train 3D CryoVesNet

This notebook trains a 3D Unet deep learning network with vesicle segmentation data. The training set is created in the [create_training](create_training.ipynb) notebook.

## Set-up

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import tensorflow as tf
import cryovesnet
#from keras import backend as K
#K.set_image_data_format('channels_last')

# import unetmic.unetmic as umic
import cryovesnet.unetmic.unetmic.unetmic as umic

In [ ]:
if tf.__version__[0] == '1':
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    tf.keras.backend.set_session(tf.Session(config=config))
    print('Check if GPU is available')
    print(tf.test.gpu_device_name())
else:
    print('Check if GPU is available2')
    print(tf.config.list_physical_devices())

In [ ]:
# TF 2Check if a GPU is available
gpu_available = tf.config.list_physical_devices('GPU')
print("Is GPU available:", len(gpu_available) > 0)

## Train 2D network Training

In [ ]:
#create the 3d-unet
# unet3d = umic.create_unet_3d()

In [ ]:
# netowrk2d = umic.create_unet_2d(128,network_name = 'eman2')

In [ ]:
folder = '/media/amin/mtwo/train_dataset_2d_128_synaptasome_1000/'
save_folder = '/media/amin/mtwo/train2d/'

#specify number of training data, validation data, batch size
numtot = 4062
numvalid = 475
batchsize = 100

In [ ]:
umic.run_training_multiGPU_2d(save_folder = save_folder, data_folder = folder, 
                              num_total = numtot,batch_size = batchsize, num_valid = numvalid,network_name="eman2")

## Create network and train CryoVesnt network Training

In [ ]:
unet3d = umic.create_unet_3d(inputsize=(32, 32, 32, 1),
                   n_depth=2,
                   n_filter_base=16,
                   kernel_size=(3, 3, 3),
                   activation='relu',
                   batch_norm=True,
                   dropout=0.1,
                   n_conv_per_depth=2,
                   pool_size=(2, 2, 2),
                   n_channel_out=1)

In [ ]:
#specifiy path to training data and to folder where to save training weights
folder = '/mnt/data/Amin/Data/train_dataset_synaptosome32/'
save_folder = '/mnt/data/Amin/Data/cryo_learning/'

#specify number of training data, validation data, batch size
numtot = 950
numvalid = 150
batchsize = 50

In [ ]:
umic.run_training(network = unet3d, save_folder = save_folder, folder = folder, 
                numtot = numtot,batchsize = batchsize, numvalid = numvalid)

## Multi GPU training

In [ ]:
import warnings
import os
warnings.filterwarnings("ignore")
os.environ["TF_CPP_MIN_LOG_LEVEL"]="2"
# Here you can choose which GPU to use
os.environ["CUDA_VISIBLE_DEVICES"]="1,2"

import tensorflow as tf
import cryovesnet
#from keras import backend as K
#K.set_image_data_format('channels_last')

# import unetmic.unetmic as umic
import cryovesnet.unetmic.unetmic.unetmic as umic

In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
# for bigger patch size 64^3
#specifiy path to training data and to folder where to save training weights
folder = '/mnt/data/Amin/Data/train_dataset_64_raw_neuron'
save_folder = '/mnt/data/Amin/Data/cryo_learning_64_nad/'

#specify number of training data, validation data, batch size
numtot = 1600
numvalid = 250
batchsize = 25

In [ ]:
umic.run_training_multiGPU(save_folder = save_folder, data_folder = folder, 
                num_total = numtot,batch_size = batchsize, num_valid = numvalid)